# Flux Balance Analysis Demo — C. autoethanogenum GEM

This notebook loads the curated iCLAU786 genome-scale model of *Clostridium autoethanogenum*,  
runs FBA for wild-type and a knockout of phosphotransacetylase (PTA),  
and compares the predicted growth rate and ethanol secretion.


In [1]:
from cobra import io
import matplotlib.pyplot as plt

# Load the SBML model
model = io.read_sbml_model("models/iCLAU786.json")


CobraSBMLError: Something went wrong reading the SBML model. Most likely the SBML model is not valid. Please check that your model is valid using the `cobra.io.sbml.validate_sbml_model` function or via the online validator at https://sbml.org/validator_servlet/ .
	`(model, errors) = validate_sbml_model(filename)`
If the model is valid and cannot be read please open an issue at https://github.com/opencobra/cobrapy/issues .

In [2]:
# Set CO uptake to -10 mmol/gDW/h (as per literature)
co_ex = model.reactions.get_by_id("EX_co_e")
co_ex.bounds = (-10, -10)


NameError: name 'model' is not defined

In [ ]:
# Run FBA for wild-type
wt_solution = model.optimize()
print("WT Growth Rate:", round(wt_solution.objective_value, 4))
print("WT Ethanol Secretion:", round(wt_solution.fluxes.get("EX_etoh_e", 0), 4))


In [ ]:
# Knock out PTA and re-run FBA
with model:
    model.reactions.get_by_id("PTA").knock_out()
    ko_solution = model.optimize()
    print("ΔPTA Growth Rate:", round(ko_solution.objective_value, 4))
    print("ΔPTA Ethanol Secretion:", round(ko_solution.fluxes.get("EX_etoh_e", 0), 4))


In [ ]:
# Plot results
labels = ["WT", "ΔPTA"]
growth = [wt_solution.objective_value, ko_solution.objective_value]
ethanol = [wt_solution.fluxes.get("EX_etoh_e", 0), ko_solution.fluxes.get("EX_etoh_e", 0)]

plt.bar(labels, growth, color='skyblue')
plt.ylabel("Growth Rate (1/h)")
plt.title("Growth Rate Comparison")
plt.show()

plt.bar(labels, ethanol, color='orange')
plt.ylabel("Ethanol Secretion Flux")
plt.title("Ethanol Secretion Comparison")
plt.show()
